In [77]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import statsmodels.formula.api as smf 
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split 
import itertools
%matplotlib inline
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_squared_error #MSE
from sklearn.metrics import make_scorer
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import time
import hdbscan
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import sklearn.cluster as cluster
from scipy.spatial.distance import cdist

%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

plot_kwds={'alpha':0.25, 's':60, 'linewidths':0}
palette = sns.color_palette('deep', 12)


In [2]:
df_clean = pd.read_csv('../01_Outputs/df_clean.csv')

In [ ]:
drop_lst = ['PROFILE_Name', 'PROFILE_Unique_Identity', 'PROFILE_Data_date',
       'PROFILE_Version_state', 'PROFILE_Data_status', 
       'PROFILE_Model', 'PROFILE_Version_name',
       'PROFILE_Trim_level', 'PROFILE_Number_of_doors', 
       'PROFILE_Vehicle_type', 'PROFILE_JATO_global_segment',
       'PROFILE_JATO_regional_segment', 'PROFILE_Model_year','PROFILE_Make','PROFILE_Body_type',
        'TARGET_Retail_price_including_delivery_charge',
        'TARGET_Retail_price',  'Segment_Competing_Nissan_Model', 
           'SPEC_Compressor_supercharger','Segment_Name','SPEC_duration (months)1007', 'SPEC_distance (km)1008', 
'SPEC_distance (km)1017', 'SPEC_duration (months)1016','SPEC_duration (months)1013', 
'SPEC_distance (km)1014','SPEC_duration (months)1010', 'SPEC_distance (km)1011'
           ]

In [ ]:
df_model = df_clean.drop(drop_lst, axis=1)

# Non Linear model with all variables -> get P-Values for feature selection

In [121]:
dummy_variables = ['SPEC_Enginetype_', 'SPEC_Wheelbase_', 'SPEC_Drivenwheels_', 'SPEC_rooftype_', 'SPEC_dutyrating_',
                  'SPEC_fueltype_','SPEC_Compressor_', 'SPEC_injection_', 'SPEC_hybridtype_', 'SPEC_Seatmaterial_',
                  'SPEC_RRseat_type_', 'SPEC_Airconditioningtype_', 'SPEC_cab_type_','SPEC_Continentgroup_', 'SPEC_CountryOfAssembly_',
                  'PROFILE_MY_','PROFILE_Make_','PROFILE_Bodytype_','Segment_']

In [62]:
# len(df)

In [60]:
# for item in dummy_variables:
#     print(item, df.filter(like=item).sum().sum())

13370

Duplicated dummy variable

In [65]:
# dupli_cols = ['SPEC_Enginetype_', 'PROFILE_MY_2024', 'PROFILE_Make_GMC', 'PROFILE_Bodytype_combi', 'Segment_NEAR LUXURY']

In [66]:
# df = df.drop(dupli_cols, axis=1)

In [163]:
df_model.columns = [col.replace('(', '').replace(')', '').\
                    replace(' ','_').replace('-','_').replace('/','_').\
                    replace('&','_').replace('.','_').replace("'", '') for col in df_model.columns]

In [164]:
dummy_lst = []
for item in dummy_variables:
    dummy_lst = list(set(dummy_lst).union(set(list(df_model.filter(like=item).columns))))

In [165]:
nondummy_lst = list(set(list(df_model.columns)) - set(dummy_lst) - set(['TARGET_Weighted_RetailPrice']))

In [166]:
item_dct = {}
for item in dummy_variables:
    item_dct[item] = ' + '.join(list(df_model.filter(like=item).columns))
for item in nondummy_lst:
    item_dct[item] = item

In [126]:
all_item_lst = list(set(dummy_lst).union(set(nondummy_lst)))

In [168]:
data = df_model.copy()

# set formula and family 
formula = "TARGET_Weighted_RetailPrice ~ 1 +" + ' + '.join(all_item_lst) 
link = sm.genmod.families.links.log()
family = sm.families.Gamma(link = link)

# Create GML model
model = smf.glm(formula=formula, data=data, family=family) 
# Fit the model
result = model.fit() 
# Get summary
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                      Generalized Linear Model Regression Results                      
=======================================================================================
Dep. Variable:     TARGET_Weighted_RetailPrice   No. Observations:                13351
Model:                                     GLM   Df Residuals:                    12332
Model Family:                            Gamma   Df Model:                         1018
Link Function:                             log   Scale:                       0.0014771
Method:                                   IRLS   Log-Likelihood:            -1.1585e+05
Date:                         Mon, 03 Jul 2023   Deviance:                       18.032
Time:                                 21:02:24   Pearson chi2:                     18.2
No. Iterations:                            100   Pseudo R-squ. (CS):              1.000
Covariance Type:                     nonrobust                                         
==========================================================================================================================
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                  1.6419      0.035     47.576      0.000       1.574       1.710
SPEC_DC1socket[T.True]                                    -0.0660      0.014     -4.636      0.000      -0.094      -0.038
SPEC_AC1socket[T.True]                                     0.0179      0.010      1.829      0.067      -0.001       0.037
SPEC_bore                                                  0.0004      0.000      1.275      0.202      -0.000       0.001
NCBS_OVERALL_Interior_Versatility_Seat_Configurability    -0.0007      0.002     -0.355      0.722      -0.005       0.003
SPEC_FRDRseat_reach_electric                               0.0792      0.030      2.634      0.008       0.020       0.138
SPEC_towingclass_II                                        0.0159      0.011      1.493      0.135      -0.005       0.037
SPEC_hands_free_release248                                -0.0110      0.003     -3.836      0.000      -0.017      -0.005
SPEC_Autonomous_drive                                     -0.0076      0.008     -0.908      0.364      -0.024       0.009
SPEC_operates_above_130_kph_78_mph                        -0.0081      0.004     -1.973      0.048      -0.016   -5.54e-05
Edmunds_Star_Rating                                       -0.0095      0.003     -3.143      0.002      -0.015      -0.004
SPEC_Opening_rear_1_4_windows                              0.0128      0.015      0.840      0.401      -0.017       0.043
SPEC_Exterior_door_handles                                 0.0659      0.010      6.818      0.000       0.047       0.085
SPEC_multifuncdisplay_numtouchscreen                      -0.0005      0.002     -0.214      0.830      -0.005       0.004
SPEC_fold_flat_to_floor842                                -0.0326      0.010     -3.348      0.001      -0.052      -0.013
SPEC_luxurytrim_instrumentpanel_chrome                    -0.0125      0.009     -1.369      0.171      -0.030       0.005
SPEC_IIHS_moderateoverlap_Marginal                        -0.0629      0.030     -2.106      0.035      -0.121      -0.004
SPEC_Parking_space_information                            -0.0133      0.005     -2.564      0.010      -0.023      -0.003
SPEC_IIHS_sideimpact_Marginal                          -4.372e-05      0.008     -0.005      0.996      -0.016       0.016
SPEC_FRPGseat_height_manual                               -0.0089      0.003     -2.678      0.007      -0.015      -0.002
SPEC_range_for_remaining_fuel                             -0.0569      0.011     -5.326      0.000      -0.078      -0.036
SPEC_luxurytrim_centerconsole_sy

In [170]:
df_pvalues = pd.DataFrame(result.pvalues,columns=['pvalues'])
significant_values = list(df_pvalues[df_pvalues['pvalues']<0.05].index)
significant_values.remove('Intercept')
significant_values.remove('SPEC_DC1socket[T.True]')
significant_values.append('SPEC_DC1socket')


In [171]:
df_model_pvalue = df_model[significant_values]
df_max_value = pd.DataFrame(df_model_pvalue.max(), columns=['max_value'])
category_vars = list(df_max_value[df_max_value['max_value'] == 1].index)
numerical_vars = list(df_max_value[df_max_value['max_value'] != 1].index)
df_model_pvalue = pd.concat([df_model_pvalue, data[['TARGET_Weighted_RetailPrice']]], axis=1)

In [176]:
pvalue_lst_cat = []
for item in category_vars:
    #define samples
    group1 = df_model_pvalue[df_model_pvalue[item]==1]
    group2 = df_model_pvalue[df_model_pvalue[item]==0]

    #perform independent two sample t-test
    result = ttest_ind(group1['TARGET_Weighted_RetailPrice'], group2['TARGET_Weighted_RetailPrice'])
    pvalue_lst_cat.append(result[1])

In [177]:
category_pvalues = pd.DataFrame({'variable':category_vars, 'pvalue':pvalue_lst_cat})

In [178]:
correlation_lst = []
pvalue_lst_num = []
for item in numerical_vars:
    #perform independent two sample t-test
    result = pearsonr(df_model_pvalue[item], df_model_pvalue['TARGET_Weighted_RetailPrice'])
    correlation_lst.append(result[0])
    pvalue_lst_num.append(result[1])

In [179]:
numerical_pvalues = pd.DataFrame({'variable':numerical_vars, 
                                  'pvalue':pvalue_lst_num,
                                  'correlation':correlation_lst})

In [180]:
numerical_pvalues

,variable,pvalue,correlation
0,Edmunds_Star_Rating,5.129746e-45,0.121405
1,SPEC_overall_length_mm,3.346218e-25,0.089587
2,SPEC_multifuncdisplay_Dashboard1_size,5.546943e-241,0.281124
3,RV_60mo,4.356938e-16,-0.070265
4,SPEC_rpm_for_maximum_torque_low,6.382878e-148,-0.221400
5,NCBS_OVERALL_Interior_Design___Layout,1.845267e-277,0.300896
6,NCBS_OVERALL_Comfort_of_the_Seats,3.384618e-306,0.315442
7,NCBS_OVERALL_Brand_Image,4.499351e-227,0.273139
8,SPEC_width_between_wheelhouses_mm,7.217065e-13,-0.062044
9,NCBS_OVERALL_Noise_Vibration_Harshness,1.012707e-241,0.281541


In [181]:
categorical = category_pvalues[category_pvalues['pvalue']<0.05]['variable'].tolist()
numerical = numerical_pvalues[(numerical_pvalues['pvalue']<0.05)&(abs(numerical_pvalues['correlation'])>0.1)]['variable'].tolist()
final_features = list(set(categorical).union(numerical))
len(final_features)

In [185]:
df_model_pvalue = df_model_pvalue[final_features+['TARGET_Weighted_RetailPrice']]

In [144]:
df_model_pvalue

,SPEC_Basepainttype_mica,SPEC_driver_monitor,SPEC_parkingassist_auto,SPEC_extdoorhundle_2tone,SPEC_Illuminated_entry_system,SPEC_fueltype_unleaded,Segment_LUXURY_FULLSIZE_SUV,SPEC_FRDRseat_reach_electric,SPEC_hands_free_release248,SPEC_Differential_lock,...,SPEC_Sunroof_electric,SPEC_Continentgroup_OTH,SPEC_multifuncdisplay_Instrument1_size,PROFILE_Bodytype_cargo_van,SPEC_published_kerb_weight_kg,SPEC_LED_lights,PROFILE_Make_AUDI,SPEC_satellite_radio,SPEC_Enginetype_HEV,TARGET_Weighted_RetailPrice
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,1930.0,1,0,1,0,44290.000000
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,1826.0,1,0,1,0,42290.000000
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,1965.0,1,0,1,0,56505.000000
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,1948.0,1,0,1,0,50565.000000
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,1861.0,1,0,1,0,54505.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13365,0,1,1,0,1,0,0,0,1,1,...,0,0,5.3,0,2119.0,1,0,1,0,49988.150685
13366,0,1,1,0,1,0,0,0,1,1,...,0,0,5.3,0,2235.0,1,0,1,0,53839.169247
13367,0,1,1,0,1,0,0,0,0,1,...,0,0,5.3,0,2119.0,1,0,1,0,49945.000000
13368,0,1,1,0,1,0,0,0,0,1,...,0,0,5.3,0,2119.0,1,0,1,0,49988.150685


In [186]:
df_model_pvalue.to_csv('df_model_1st.csv', index=False)